# Capstone Project- Data Collection Workbook
Cary Mosley, May 2020

## Import Libraries

In [3]:
import pandas as pd
import calendar, time
import datetime as dt
from datetime import date
import praw
import requests
from requests import get
import json
import quandl






## Collect Data

There is dada starting June 30th, 2006 for each of our indicators so the time frame I plan to use is from 2006-06-30 through 2020-04-30

In [4]:
#Set date window to collect data

#Stock data interval
interval='1d'

#Set dates in YYYY-MM-DD format
end_date = '2020-05-05'
start_date = '2006-06-30'


#Stock Tickers
tickers = ['SPY','%5EVIX','IWM','QQQ']

### Collect historical stock data from Yahoo Finance

In [5]:
#Get Historical StockData From Yahoo Finance
def StockData(ticker, start_date, end_date = date.today(), interval = '1d'):
    start_date_epoch = calendar.timegm(time.strptime(str(start_date), '%Y-%m-%d'))
    end_date_epoch = calendar.timegm(time.strptime(str(end_date), '%Y-%m-%d'))
    url = 'https://query1.finance.yahoo.com/v7/finance/download/' + ticker
    url = url + '?period1=' + str(start_date_epoch) + '&period2=' + str(end_date_epoch)
    url = url + '&interval=' + interval + '&events=history'
    df = pd.read_csv(url)
    df.sort_values(by=['Date'],ascending=False,inplace=True)
    df.set_index('Date', inplace=True)
    columns_to_drop = ['Open','High','Low','Close','Volume']
    df.drop(columns=columns_to_drop,inplace=True)
    return df

In [6]:
#Create and export SPY prices
ticker=tickers[0]

spy_df = StockData(ticker, start_date, end_date, interval)
spy_df.head()



,Adj Close
Date,
2020-05-04,283.570007
2020-05-01,282.790009
2020-04-30,290.480011
2020-04-29,293.209991
2020-04-28,285.730011


In [7]:
#Export to csv
spy_df.to_csv('Data/spy.csv')

In [8]:
#Create and export VIX prices
ticker=tickers[1]

vix_df = StockData(ticker, start_date, end_date, interval)
vix_df.head()


,Adj Close
Date,
2020-05-04,35.970001
2020-05-01,37.189999
2020-04-30,34.150002
2020-04-29,31.230000
2020-04-28,33.570000


In [9]:
#Export to csv
vix_df.to_csv('Data/vix.csv')

In [10]:
#Create and export IWM prices
ticker=tickers[2]

iwm_df = StockData(ticker, start_date, end_date, interval)
iwm_df.head()


,Adj Close
Date,
2020-05-04,125.680000
2020-05-01,125.139999
2020-04-30,130.309998
2020-04-29,135.460007
2020-04-28,129.110001


In [11]:
#Export to csv
iwm_df.to_csv('Data/iwm.csv')

In [12]:
#Create and export QQQ prices
ticker=tickers[3]

qqq_df = StockData(ticker, start_date, end_date, interval)
qqq_df.head()


,Adj Close
Date,
2020-05-04,215.220001
2020-05-01,212.740005
2020-04-30,218.910004
2020-04-29,219.000000
2020-04-28,211.500000


In [13]:
#Export to csv
qqq_df.to_csv('Data/qqq.csv')

### Scrape reddit submissions
This section was not used for the final project

In [29]:
#Use PRAW to perform API pull
with open('reddit.json') as f:
    reddit_keys= json.load(f)

reddit = praw.Reddit(client_id = reddit_keys['client_id'], client_secret = reddit_keys['client_secret'], 
                     password = reddit_keys['password'], user_agent = reddit_keys['user_agent'],
                     username = reddit_keys['username'])


In [30]:
#Choose subreddit
subreddit = 'WallStreetBets'
#Use praw to collect data
top_posts = reddit.subreddit(subreddit).top(limit=500)
#create dictionary of elements that are returned
topics_dict = { "title":[], 
                "score":[], 
                "id":[], "url":[],  
                "comms_num": [], 
                "created": [], 
                "body":[]}

#create dataframe from the praw data
for post in top_posts:
    topics_dict["title"].append(post.title)
    topics_dict["score"].append(post.score)
    topics_dict["id"].append(post.id)
    topics_dict["url"].append(post.url)
    topics_dict["comms_num"].append(post.num_comments)
    topics_dict["created"].append(post.created)
    topics_dict["body"].append(post.selftext)
topics_data = pd.DataFrame(topics_dict)

In [31]:
topics_data.head()

,title,score,id,url,comms_num,created,body
0,"Elon has transcended time, space, and county r...",69256,ghw12u,https://i.redd.it/frcjfc2967y41.jpg,4941,1.589258e+09,
1,How to get oil back up,58582,g55or2,https://i.redd.it/w5iqqihjo2u41.jpg,548,1.587461e+09,
2,All In The War Machine,55123,eld4i6,https://i.redd.it/cj2vw26nmd941.jpg,784,1.578440e+09,
3,Type yy into google.,52812,c75d5x,https://www.reddit.com/r/wallstreetbets/commen...,17263,1.561875e+09,That’s all I’m saying.
4,Oil is now expenzive,49777,d51f4o,https://i.redd.it/2j386s5iuym31.png,1018,1.568673e+09,


In [32]:
#Convert created to date-time
topics_data['created'] = topics_data["created"].apply(lambda x: dt.datetime.fromtimestamp(x))

In [33]:
topics_data.head()

,title,score,id,url,comms_num,created,body
0,"Elon has transcended time, space, and county r...",69256,ghw12u,https://i.redd.it/frcjfc2967y41.jpg,4941,2020-05-12 00:41:17,
1,How to get oil back up,58582,g55or2,https://i.redd.it/w5iqqihjo2u41.jpg,548,2020-04-21 05:18:21,
2,All In The War Machine,55123,eld4i6,https://i.redd.it/cj2vw26nmd941.jpg,784,2020-01-07 18:40:30,
3,Type yy into google.,52812,c75d5x,https://www.reddit.com/r/wallstreetbets/commen...,17263,2019-06-30 02:14:57,That’s all I’m saying.
4,Oil is now expenzive,49777,d51f4o,https://i.redd.it/2j386s5iuym31.png,1018,2019-09-16 18:28:12,


### Use Quandl API to scrape data

In [9]:
#Import quandl API key
with open('quandl.json') as f:
    quandl_keys= json.load(f)
quandl.ApiConfig.api_key = quandl_keys['api_key']

#### American Association of Individual Investor’s sentiment data

In [26]:
#AAII Data API Pull from Quandl
AAII_data = 'AAII/AAII_SENTIMENT'
AAII_sentiment_df = quandl.get(AAII_data, start_date=start_date, end_date=end_date)

In [27]:

AAII_sentiment_df.head()


,Bullish,Neutral,Bearish,Total,Bullish 8-Week Mov Avg,Bull-Bear Spread,Bullish Average,Bullish Average + St. Dev,Bullish Average - St. Dev,S&P 500 Weekly High,S&P 500 Weekly Low,S&P 500 Weekly Close
Date,,,,,,,,,,,,
2006-06-01,0.3077,0.1923,0.5000,1.0000,0.406488,-0.1923,0.384907,0.488293,0.281522,1285.71,1269.19,1285.71
2006-06-08,0.2623,0.2869,0.4508,1.0000,0.382575,-0.1885,0.384907,0.488293,0.281522,1257.07,1249.43,1249.72
2006-06-15,0.2641,0.1861,0.5498,1.0000,0.373425,-0.2857,0.384907,0.488293,0.281522,1243.59,1230.01,1243.04
2006-06-22,0.3440,0.2400,0.4160,1.0000,0.363787,-0.0720,0.384907,0.488293,0.281522,1251.92,1241.53,1244.70
2006-06-29,0.3860,0.2164,0.3977,1.0001,0.354675,-0.0117,0.384907,0.488293,0.281522,1257.29,1245.94,1255.88


In [28]:
AAII_sentiment_df.columns

Index(['Bullish', 'Neutral', 'Bearish', 'Total', 'Bullish 8-Week Mov Avg',
       'Bull-Bear Spread', 'Bullish Average', 'Bullish Average + St. Dev',
       'Bullish Average - St. Dev', 'S&P 500 Weekly High',
       'S&P 500 Weekly Low', 'S&P 500 Weekly Close'],
      dtype='object')

In [29]:
#Drop Columns
drop_columns = ['Total', 'Bullish Average','Bullish 8-Week Mov Avg', 'Bullish Average + St. Dev',
       'Bullish Average - St. Dev','S&P 500 Weekly High','S&P 500 Weekly Low','S&P 500 Weekly Close']

In [30]:
AAII_sentiment_df.drop(columns=drop_columns,inplace=True)

In [31]:
AAII_sentiment_df.head()

,Bullish,Neutral,Bearish,Bull-Bear Spread
Date,,,,
2006-06-01,0.3077,0.1923,0.5000,-0.1923
2006-06-08,0.2623,0.2869,0.4508,-0.1885
2006-06-15,0.2641,0.1861,0.5498,-0.2857
2006-06-22,0.3440,0.2400,0.4160,-0.0720
2006-06-29,0.3860,0.2164,0.3977,-0.0117


In [32]:
#Export to CSV
AAII_sentiment_df.to_csv('Data/AAII.csv')

#### FINRA Short Interest By Security 
This was not used for the final project

In [71]:
#Get Finra Data 
def finra_data(ticker,start_date, end_date = date.today(), exchange='NYSE'):
    finra_short = 'FINRA/FN'
    finra_dict = {'NYSE': 'YX','NASDAQ':'SQ'}
    if exchange == 'NYSE':
        finra_call = finra_short + 'YX_' + ticker
        print(finra_call)
        return quandl.get(finra_call, start_date=start_date, end_date=end_date)
    elif exchange == 'NASDAQ':
        finra_call = finra_short + 'SQ_' + ticker
        print(finra_call)
        return quandl.get(finra_call, start_date=start_date, end_date=end_date)
    else:
        print('error')
        return 'error'
    return

In [72]:
exchange = 'NYSE'
short_df = finra_data(ticker,start_date,end_date,exchange)

FINRA/FNYX_SPY


In [73]:
short_df

,ShortVolume,ShortExemptVolume,TotalVolume
Date,,,
2015-05-12,2111990.0,0.0,2681572.0
2015-05-13,1278698.0,0.0,1830248.0
2015-05-14,1235013.0,0.0,1652625.0
2015-05-15,1269385.0,0.0,1967962.0
2015-05-18,846827.0,0.0,1309836.0
...,...,...,...
2020-05-05,2980150.0,10.0,5148326.0
2020-05-06,3298070.0,255.0,4846833.0
2020-05-07,3652096.0,0.0,5451842.0


#### NAAIM Exposure Index

In [25]:
#NAAIM Data API Pull from Quandl
exposure = 'NAAIM/NAAIM'
NAAIM_exposure_df = quandl.get(exposure, start_date=start_date, end_date=end_date)


In [18]:
NAAIM_exposure_df.head()

,Mean/Average,Most Bearish Response,Quart 1 (25% at/below),Quart 2 (median),Quart 3 (25% at/above),Most Bullish Response,Standard Deviation,NAAIM Number,S&P 500
Date,,,,,,,,,
2006-06-30,56.33,-125.0,40.0,65.0,100.0,125.0,53.51,56.33,1326.05
2006-07-05,19.44,-100.0,0.0,20.0,50.0,100.0,55.55,19.44,1265.48
2006-07-12,31.20,-50.0,0.0,25.0,50.0,150.0,47.84,31.20,1236.20
2006-07-19,18.76,-100.0,0.0,25.0,50.0,100.0,38.17,18.76,1240.29
2006-07-26,17.38,-50.0,0.0,15.0,50.0,100.0,33.78,17.38,1278.55


In [19]:
NAAIM_exposure_df.columns

Index(['Mean/Average', 'Most Bearish Response', 'Quart 1 (25% at/below)',
       'Quart 2 (median)', 'Quart 3 (25% at/above)', 'Most Bullish Response',
       'Standard Deviation', 'NAAIM Number', 'S&P 500'],
      dtype='object')

In [20]:
#drop columns
drop_columns = ['Quart 1 (25% at/below)','Quart 3 (25% at/above)', 'Standard Deviation', 'NAAIM Number', 'S&P 500']

In [21]:
NAAIM_exposure_df.drop(columns=drop_columns,inplace=True)

In [22]:
NAAIM_exposure_df.head()

,Mean/Average,Most Bearish Response,Quart 2 (median),Most Bullish Response
Date,,,,
2006-06-30,56.33,-125.0,65.0,125.0
2006-07-05,19.44,-100.0,20.0,100.0
2006-07-12,31.20,-50.0,25.0,150.0
2006-07-19,18.76,-100.0,25.0,100.0
2006-07-26,17.38,-50.0,15.0,100.0


In [23]:
#Export to CSV
NAAIM_exposure_df.to_csv('Data/NAAIM.csv')

#### NYTimes Articles

In [21]:
#Import NYTimes API key
with open('nytimes.json') as f:
    nyt_keys= json.load(f)


I had to break the API pulls into chunks as it was 1-2m articles and appending to one giant dataframe took too long. I'll be merging and filtering the articles in the DataCleanand Process workbook and combining into one dataframe there.

In [23]:
#Set year and month ranges
years = range(2006,2021)
months1 = range(1,6)
months2 = range(7,13)

The loop below pulls the publication date, headline and snippet from all NY Times articles over the specified time frame and exports to a csv

In [22]:
#loop through each year
for year in years:
    #create temp dataframe
    nyt_df = pd.DataFrame(columns=['Date', 'Headline','Snippet'])
    
    #loop through first 6 months
    for month in months1:
        #set the url for the api pull
        url = 'https://api.nytimes.com/svc/archive/v1/'
        url = url + str(year) + '/' + str(month) + '.json?api-key=' + nyt_keys['api_key']
        
        #pull data from NY Times
        responses=requests.get(url)
        articles = responses.json()
        print(year,month)
        
        #loop through each article and grab the relavent fields
        for article in articles['response']['docs']:
            try:
                nyt_df = nyt_df.append({'Date' : article['pub_date'] , 'Headline' : article['headline']['main'] ,'Snippet' : article['snippet']},ignore_index=True)
            except:
                print('bad entry')
                continue
    #set path to export
    path = 'Data/nyt'+str(year)+'1.csv'  
    
    #export the temp dataframe to a csv
    nyt_df.to_csv(path)
    
    #recreate new temp dataframe
    nyt_df = pd.DataFrame(columns=['Date', 'Headline','Snippet'])
   
    #loop through second six months
    for month in months2:
        #set the url for the api pull
        url = 'https://api.nytimes.com/svc/archive/v1/'
        url = url + str(year) + '/' + str(month) + '.json?api-key=' + nyt_keys['api_key']
        
        #pull data from NY Times
        responses=requests.get(url)
        articles = responses.json()
        print(year,month)
        
        #loop through each article and grab the relavent fields
        for article in articles['response']['docs']:
            try:
                nyt_df = nyt_df.append({'Date' : article['pub_date'] , 'Headline' : article['headline']['main'] ,'Snippet' : article['snippet']},ignore_index=True)
            except:
                print('bad entry')
                continue
    #set path to export
    path = 'Data/nyt'+str(year)+'2.csv'   
    
    #export the temp dataframe to a csv
    nyt_df.to_csv(path)

2011 1
2011 2
2011 3
2011 4
2011 5
2011 7
2011 8
2011 9
2011 10
2011 11
2011 12
bad entry
